<div align="center" style="display: flex; align-items: center; justify-content: left;">
  <img src="../assets/weco.svg" alt="WeCo AI" style="height: 50px; margin-right: 10px;">
  <a href="https://git.io/typing-svg"><img src="https://readme-typing-svg.demolab.com?font=Georgia&size=32&duration=4000&pause=400&color=FD4578&vCenter=true&multiline=false&width=750&height=100&lines=AI+Function+Builder;" alt="Typing SVG" /></a>
</div>

## Getting Started

`weco` is a client facing API for interacting with the [WeCo AI](https://www.weco.ai/) function builder [service](https://weco-app.vercel.app/function). Use this API to build *complex* systems *fast* $f$(👷‍♂️)!

Here's a short introduction to how you can use our client. Feel free to follow along:

<a href="https://colab.research.google.com/github/WecoAI/weco-python/blob/main/examples/cookbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<a target="_blank" href="https://lightning.ai/new?repo_url=https%3A%2F%2Fgithub.com%2FWecoAI%2Fweco-python%2Fblob%2Fmain%2Fexamples%2Fcookbook.ipynb"><img src="https://pl-bolts-doc-images.s3.us-east-2.amazonaws.com/app-2/studio-badge.svg" alt="Open in Studio" width=100 height=20/></a>

Install the `weco` package in your terminal of choice:

In [ ]:
%pip install weco

Export your API key which can be found [here](https://weco-app.vercel.app/account).

In [ ]:
%env WECO_API_KEY=<YOUR_WECO_API_KEY>

You can build powerful AI functions for complex tasks quickly and without friction. For example, you can create a function in the [web console](https://weco-app.vercel.app/function) with this description:

> "Analyze a business idea and provide a structured evaluation. Output a JSON with 'viability_score' (0-100), 'strengths' (list), 'weaknesses' (list), and 'next_steps' (list)."

Once created, you can query this function anywhere in your code with just a few lines:

In [ ]:
from weco import query

response = query(
    fn_name="BusinessIdeaAnalyzer-XYZ123",  # Replace with your actual function name
    text_input="A subscription service for personalized, AI-generated bedtime stories for children."
)

print(response)

## Multimodality

Our AI functions can interpret complex visual information, follow instructions in natural language and provide practical insights. Let's explore how we can all have a chef give us personalized advice! They can look at the food we have and offer recipe suggestions even providing nutritional information.
As shown in the example above, you can provde the image input in various ways such as:
1. Base64 encoding
3. Public URL
4. Local Path

In [ ]:
from weco import build, query
import base64

task_description = """
Create a Smart Home Energy Analyzer that can process images of smart meters, home exteriors, 
and indoor spaces to provide energy efficiency insights. The analyzer should:
    1. Interpret smart meter readings
    2. Assess home features relevant to energy consumption
    3. Analyze thermostat settings
    4. Provide energy-saving recommendations
    5. Evaluate renewable energy potential

The output should include:
    - 'energy_consumption': current usage and comparison to average
    - 'home_analysis': visible energy features and potential issues
    - 'thermostat_settings': current settings and recommendations
    - 'energy_saving_recommendations': actionable suggestions with estimated savings
    - 'renewable_energy_potential': assessment of current and potential renewable energy use
    - 'estimated_carbon_footprint': current footprint and potential reduction
"""

fn_name, _ = build(task_description=task_description)

request = """
Analyze these images of my home and smart meter to provide energy efficiency insights 
and recommendations for reducing my electricity consumption.
"""

# Base64 encoded image
with open("/path/to/home_exterior.jpeg", "rb") as img_file:
    my_home_exterior = base64.b64encode(img_file.read()).decode('utf-8')

query_response = query(
    fn_name=fn_name,
    text_input=request,
    images_input=[
        "https://example.com/my_smart_meter_reading.png",  # Public URL
        f"data:image/jpeg;base64,{my_home_exterior}",      # Base64 encoding
        "/path/to/living_room_thermostat.jpg"              # Local image path
    ]
)

for key, value in query_response["output"].items(): print(f"{key}: {value}")
print(f"Input Tokens: {query_response['in_tokens']}")
print(f"Output Tokens: {query_response['out_tokens']}")
print(f"Latency: {query_response['latency_ms']} ms")

## Running Example

Consider the previous example of:
> "I want to evaluate the feasibility of a machine learning task. Help me understand this through - 'feasibility', 'justification', and 'suggestions'."

Here's how you can take advantage of our API to best suit your needs.

In [ ]:
task_description = "I want to evaluate the feasibility of a machine learning task. Give me a json object with three keys - 'feasibility', 'justification', and 'suggestions'."

## Dense vs. Sparse Usage

Though we recommend building functions in the [web console](https://weco-app.vercel.app/function) for maximum control over the function I/O, you can also do this programmatically:

In [ ]:
from weco import build, query

# Describe the task you want the function to perform
fn_name, fn_desc = build(task_description=task_description)
print(f"AI Function {fn_name} built. This does the following - \n{fn_desc}.")

# Query the function with a specific input
query_response = query(
    fn_name=fn_name,
    text_input="I want to train a model to predict house prices using the Boston Housing dataset hosted on Kaggle."
)
for key, value in query_response["output"].items(): print(f"{key}: {value}")
print(f"Input Tokens: {query_response['in_tokens']}")
print(f"Output Tokens: {query_response['out_tokens']}")
print(f"Latency: {query_response['latency_ms']} ms")

We recommend to use the `weco.build` and `weco.query` when you want to build or query LLM functions sparsely, i.e., you **don't** call `weco.build` or `weco.query` in many places within your code. However, for more dense usage, we've found users prefer our `weco.WecoAI` client instance. Its easy to switch between the two as shown below:

In [ ]:
from weco import WecoAI

# Connect to our service, using our client
client = WecoAI()

# Make the same query as before
query_response = client.query(
    fn_name=fn_name,
    text_input="I want to train a model to predict house prices using the Boston Housing dataset hosted on Kaggle."
)
for key, value in query_response.items(): print(f"{key}: {value}")

## Batching

We understand that sometimes, independent of how many times in your code you call `weco` functions, you want to submit a large batch of requests for the same or different LLM functions you've created. This can be done in the following ways:

In [ ]:
from weco import batch_query

# Query the same function with multiple inputs by batching them for maximum efficiency
input_1 = {"text_input": "I want to train a model to predict house prices using the Boston Housing dataset hosted on Kaggle."}
input_2 = {
    "text_input": "I want to train a model to classify digits using the MNIST dataset hosted on Kaggle using a Google Colab notebook. Attached is an example of what some of the digits would look like.",
    "images_input": ["https://machinelearningmastery.com/wp-content/uploads/2019/02/Plot-of-a-Subset-of-Images-from-the-MNIST-Dataset-1024x768.png"]
}
query_responses = batch_query(
    fn_names=fn_name,
    batch_inputs=[input_1, input_2]
)
for i, query_response in enumerate(query_responses):
    print("-"*50)
    print(f"For input {i + 1}")
    for key, value in query_response["output"].items(): print(f"{key}: {value}")
    print("-"*50)

You can do the same using the `weco.WecoAI` client. If you wanted to batch different functions, you can pass a list of function names to the `batch_query()` function. Note that the names of functions would need to be ordered the same as the function inputs provided.

In addition, `weco.batch_query` takes the input batch as an array of individual inputs formatted in the following way -
```json
{
    "text_input": "Your text input",
    "images_input": ["image1", "image2"]
}
```

## Async Calls

Until now you've been making synchronous calls to our client by we also support asynchronous programmers. This is actually how we implement batching! You can also make asynchronous calls to our service using our `weco.WecoAI` client or as shown below for the same example as before:

In [ ]:
from weco import abuild, aquery

# Describe the task you want the function to perform
fn_name, fn_desc = await abuild(task_description=task_description)
print(f"AI Function {fn_name} built. This does the following - \n{fn_desc}.")

# Query the function with a specific input
query_response = await aquery(
    fn_name=fn_name,
    text_input="I want to train a model to predict house prices using the Boston Housing dataset hosted on Kaggle."
)
for key, value in query_response["output"].items(): print(f"{key}: {value}")
print(f"Input Tokens: {query_response['in_tokens']}")
print(f"Output Tokens: {query_response['out_tokens']}")
print(f"Latency: {query_response['latency_ms']} ms")

## Interpretability

You can now understand why a model generated an output simply by passing `return_reasoning=True` at query time!

In [ ]:
from weco import build, query

# Describe the task you want the function to perform
fn_name, fn_desc = build(task_description=task_description)
print(f"AI Function {fn_name} built. This does the following - \n{fn_desc}.")

# Query the function with a specific input
query_response = query(
    fn_name=fn_name,
    text_input="I want to train a model to predict house prices using the Boston Housing dataset hosted on Kaggle.",
    return_reasoning=True
)
for key, value in query_response["output"].items(): print(f"{key}: {value}")
for i, step in enumerate(query_response["reasoning_steps"]): print(f"Step {i+1}: {step}")
print(f"Input Tokens: {query_response['in_tokens']}")
print(f"Output Tokens: {query_response['out_tokens']}")
print(f"Latency: {query_response['latency_ms']} ms")

## Happy Building $f$(👷‍♂️)!